In [1]:
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns 
import matplotlib.pyplot as plt
import math
import scipy.signal as sig

In [2]:
## calculate change in hip Y position as a proxy for velocity in 2D vertical videos 

In [3]:
def calc_pix_size_change(start_frame, end_frame, df, fps): 
    start_row = df.loc[df['frame'] == start_frame]
    end_row = df.loc[df['frame'] == end_frame]

    # height in pixel at start and end of time group 
    pix_h_start = start_row['approx_height_Y_pix_smooth'].iloc[0]
    pix_h_end = end_row['approx_height_Y_pix_smooth'].iloc[0]

    # for consistency, only calculate when walking toward camera - height is getting bigger as walking toward 
    if pix_h_end > pix_h_start:
        # change in pixel height relative to start
        delta_pix_h_rel = delta_pix_h / pix_h_start
    else: 
        delta_pix_h_rel = np.nan

    return delta_pix_h_rel

In [4]:
def calc_stride_time_proxy(yolo_df, video_id_date_name, dir_out_prefix, fps): 

    # left hip as proxy for "height" smooth left hip position 
    l_hip_yolo_df = yolo_df.loc[yolo_df['label'] == 'left_hip']
    l_hip_yolo_df.set_index('frame', inplace=True)

    l_hip_yolo_df = l_hip_yolo_df.copy()
    l_hip_yolo_df.loc[:, 'approx_height_Y_pix'] = abs(l_eye_yolo_df['Y_yolo']) 
    l_hip_yolo_df.loc[:, 'approx_height_Y_pix_smooth'] = l_hip_yolo_df['approx_height_Y_pix'].rolling(window=25, min_periods = 1).mean()
            
    # drop X columns 
    height_df = l_hip_yolo_df.drop(columns = ['X_yolo'])
    height_df.reset_index(inplace=True)

    # add "time_group" label column, will use to group 
    height_df['time_group'] = height_df['frame'] / (fps)
    height_df['time_group'] = height_df['time_group'].apply(math.floor)
    
    # identify peaks and valleys in pixel height (approximate turn locations) 
    peaks, _ = sig.find_peaks(height_df['approx_height_Y_pix_smooth'], distance = fps)
    valleys, _ = sig.find_peaks(-height_df['approx_height_Y_pix_smooth'], distance = fps)
    peak_frames = height_df.iloc[peaks]['frame']
    valley_frames = height_df.iloc[valleys]['frame']
    peaks_valleys = np.concatenate((peak_frames, valley_frames))

    # plot height in pixels with time groups and peaks valleys 
    sns.scatterplot(x = 'frame', y = 'approx_height_Y_pix_smooth', data = height_df, label = 'approx_height_Y_pix_smooth')
    plt.plot(height_df['frame'], height_df['approx_height_Y_pix_smooth'], label='Smoothed Height')
    plt.scatter(peak_frames, height_df.iloc[peaks]['approx_height_Y_pix_smooth'], color='red', label='Peaks')
    plt.scatter(valley_frames, height_df.iloc[valleys]['approx_height_Y_pix_smooth'], color='blue', label='Valleys')
    for time_group in height_df['time_group'].unique():
        plt.axvline(x = time_group * (fps), color = 'grey', alpha = 0.5)
    
    # plot output folders 
    #save outputs 
    output_folder = os.path.join(output_parent_folder, '005_gait_metrics', 'vel_pixel_proxy')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    vid_in_path_no_ext = os.path.splitext(os.path.basename(vid_in_path))[0]

    plt.title(vid_in_path + ' ' + name)
    fig_path = os.path.join(output_folder, vid_in_path_no_ext + '_pix_height.png') 
    plt.savefig(fig_path) 
    plt.close()

    # calculate depth proxy values and summarize 
    depth_proxies_all = []

    for current_sec_group in height_df['time_group'].unique():
        current_height_df = height_df.loc[height_df['time_group'] == current_sec_group]

        current_start_frame = current_height_df['frame'].iloc[0]
        current_end_frame = current_height_df['frame'].iloc[-1]

        # if this current time_group group contains peak or valley, skip because likeley a turn 
        if current_height_df.loc[current_start_frame: current_end_frame, 'frame'].isin(peaks_valleys).any():
            # print('Skipped - contains peak or valley') 
            delta_pix_h_rel = np.nan
        else: 
            delta_pix_h_rel  = calc_pix_size_change(current_start_frame, current_end_frame, height_df, fps)
                
        # combine all time groups into one array 
        depth_proxies_all.append({'start_frame' : current_start_frame,
                                  'delta_pix_h_rel' : delta_pix_h_rel})

                             
    # convert array to df 
    depth_proxies_all_df = pd.DataFrame(depth_proxies_all)
    depth_proxies_all_df = depth_proxies_all_df.replace([np.inf, -np.inf], np.nan) # replace inf with nan
    depth_proxies_all_df.to_csv(os.path.join(output_folder, vid_in_path_no_ext + '_pix_height.csv')) 
    
    # calculate median relative change in pixels 
    delta_pix_h_rel_median = depth_proxies_all['delta_pix_h_rel'].median()

    return(delta_pix_h_rel_median) 

In [5]:
## convert to .py file so functions can be used in other scripts 
!jupyter nbconvert --to script vel_pixel_h_proxy.ipynb

[NbConvertApp] Converting notebook vel_pixel_h_proxy.ipynb to script
[NbConvertApp] Writing 4998 bytes to vel_pixel_h_proxy.py
